In [4]:
df = pd.read_csv('kongres.csv')
df = df.T.reset_index()
df = df.rename(columns = {'index' : 'names'})
df["names"].to_csv('kongress.csv', index=False)

In [3]:
import pandas as pd

In [1]:
channels = open("kongres.csv", "r").read().split("\n")

In [ ]:
#IN REQUESTS
import requests
import requests.auth
import json


def assign(key, name):
    url = https://dev.pushshift.io:9200/telegram/_search
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    data = json.dumps({"fields": {"assignee": {"name": name}}})

    r = requests.put(url, data=data, headers=headers, auth=requests.auth.HTTPBasicAuth('fred', 'fred'))

    print(r.status_code)
    print(r.json())

In [ ]:
{
  "query": {
    "simple_query_string" : {
        "query": "\"fried eggs\" +(eggplant | potato) -frittata",
        "fields":"message",
        ”size”:1
  }
}

In [ ]:
#MATCH QUERY
{
  "query": {
    "match": {
      "message": {
        "query": "Навальн"
      }
    }
  }
}

In [ ]:
#DATERANGE

{
  "query": {
    "range": {
      "date": {
        "gte": 1609455600,
        "boost": 2.0
      }
    }
  }
}

In [ ]:
#ORDERDESC + QUERY
{    
    "sort" : [
        { "date" : {"order" : "desc"}},
    ],
    "query" : {
        "term" : { "message" : "Наваль" }
    }
}

In [ ]:
#SPECIFY ANALYZER
{
  "query": {
    "match": {
      "message": {
        "query": "Quick foxes",
        "analyzer": "stop"
      }
    }
  }
}

In [47]:
import requests
import json

def make_query(url, q, alg, field, shard_size=10000, size=25):
    """See this gist for docs"""
    query = {"query" : { "match" : {field : q } },
             "size": 0,
             "aggs" : {
                 "my_sample" : {
                     "sampler" : {"shard_size" : shard_size},
                     "aggs": {
                        "keywords" : {
                            "significant_text" : {
                                "size": size,
                                "field" : field,
                                alg:{}
                             }
                        }
                    }
                }
            }
        }
    # return (f'{url}/_search', data=json.dumps(query), headers={'Content-Type':'application/json'}).json()
    return [row['key'] for row in requests.post(f'{url}/_search', data=json.dumps(query),
                                                headers={'Content-Type':'application/json'}).json()['aggregations']['my_sample']['keywords']['buckets']]

In [42]:
import requests
import json

def make_query(url, q, alg, field, shard_size=10000, size=25):
    """See this gist for docs"""
    query = {"query" : { "match" : {field : q } },
             "size": 0,
             "aggs" : {
                 "my_sample" : {
                     "sampler" : {"shard_size" : shard_size},
                     "aggs": {
                        "keywords" : {
                            "significant_text" : {
                                "size": size,
                                "field" : field,
                                alg:{}
                             }
                        }
                    }
                }
            }
        }
    return (f'{url}/_search', json.dumps(query), {'Content-Type':'application/json'})

In [48]:
make_query("http://localhost:9200", "congress", "mutual_information ","message", 5000, 25)

KeyError: 'aggregations'

In [65]:
# Author : Rajathithan Rajasekar
# Date: 05/30/2020
# Retrieve data from elasticsearch index

from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np
import json

# create a client instance of the library
es = Elasticsearch(
    hosts = [{'host':'localhost', 'port': '9200'}],
    http_auth=("", ""),
    )
#écrire la query ici en DSL

terms ={
  "query": {
    "match": {
      "message": {
        "query": "Навальный"
      }
    }
  }
}
total_docs = 50000
response = es.search(
    index='telegram',
    body=terms,
    size=total_docs
)

print(es.info())

elastic_docs = response["hits"]["hits"]

fields = {}
for num, doc in enumerate(elastic_docs):
    source_data = doc["_source"]
    for key, val in source_data.items():
        try:
            fields[key] = np.append(fields[key], val)
        except KeyError:
            fields[key] = np.array([val])

traffic_data = pd.DataFrame(fields)
print(traffic_data.info())

{'name': 'laptop1', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'Shyq4SDwRSesLXAYIuxRdw', 'version': {'number': '7.10.1', 'build_flavor': 'unknown', 'build_type': 'unknown', 'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa', 'build_date': '2020-12-05T01:00:33.671820Z', 'build_snapshot': False, 'lucene_version': '8.7.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel_id     10 non-null     int32 
 1   channel_name   10 non-null     object
 2   date           10 non-null     int32 
 3   grouped_id     0 non-null      object
 4   id             10 non-null     int64 
 5   message        10 non-null     object
 6   message_id     10 non-null     int32 
 7   post           10 non-null     bool

In [21]:
traffic_data.head(-5)

,channel_id,channel_name,date,grouped_id,id,message,message_id,post,post_author,retrieved_utc,silent,updated_utc,via_bot_id
0,1217708678,global_traders_insider_trading_c,1610618460,None,5230018948066102702,EUR/USD. January 14. COT report. The House of ...,708014,True,None,1610626419,False,1610626419,None
1,1217708678,global_traders_insider_trading_c,1610592681,None,5230018948066102532,FirstSquawk Just tweeted *********************...,707844,True,None,1610626419,False,1610626419,None
2,1217708678,global_traders_insider_trading_c,1610592149,None,5230018948066102522,{{picture}} Mollie Hemingway on Trump impeachm...,707834,True,None,1610626419,False,1610626419,None
3,1459513255,loomeredofficial,1610556942,None,6268561698303518827,"In another update about the end of the world, ...",10347,True,None,1610573848,False,1610573848,None
4,1269687252,DeccanHeraldNews,1610556316,None,5453265223488129977,"Centre misleading country, Supreme Court on fa...",19385,True,None,1610568489,False,1610568489,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,1062501411,QueryHome,1561454895,None,4563408812198885382,Who was the Viceroy of British India at the ti...,30726,True,QueryHome,1610575061,False,1610575061,None
9991,1155956626,ABP_News,1561449003,None,4964795904264518887,"PM Modi, Shah remember 1975 emergency as 'dark...",15591,True,None,1610625647,False,1610625647,None
9992,1217708678,global_traders_insider_trading_c,1561447072,None,5230018948065599989,SC rejects Congress#39; plea against EC decisi...,205301,True,None,1610630636,False,1610630636,None
9993,1217708678,global_traders_insider_trading_c,1561446300,None,5230018948065599969,SC refuses to entertain Gujarat Congress plea ...,205281,True,None,1610630636,False,1610630636,None


In [25]:
es.indices.put_settings(index="telegram",
                        body= {"index" : {
                                "max_result_window" : 500 000
                              }})

{'acknowledged': True}

In [22]:
import datetime
timestamp = datetime.datetime.fromtimestamp(1561445077)
print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

2019-06-25 08:44:37


In [23]:
traffic_data[traffic_data['date']>=1609455600]

,channel_id,channel_name,date,grouped_id,id,message,message_id,post,post_author,retrieved_utc,silent,updated_utc,via_bot_id
0,1217708678,global_traders_insider_trading_c,1610618460,None,5230018948066102702,EUR/USD. January 14. COT report. The House of ...,708014,True,None,1610626419,False,1610626419,None
1,1217708678,global_traders_insider_trading_c,1610592681,None,5230018948066102532,FirstSquawk Just tweeted *********************...,707844,True,None,1610626419,False,1610626419,None
2,1217708678,global_traders_insider_trading_c,1610592149,None,5230018948066102522,{{picture}} Mollie Hemingway on Trump impeachm...,707834,True,None,1610626419,False,1610626419,None
3,1459513255,loomeredofficial,1610556942,None,6268561698303518827,"In another update about the end of the world, ...",10347,True,None,1610573848,False,1610573848,None
4,1269687252,DeccanHeraldNews,1610556316,None,5453265223488129977,"Centre misleading country, Supreme Court on fa...",19385,True,None,1610568489,False,1610568489,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1217708678,global_traders_insider_trading_c,1609532606,None,5230018948066096680,Congress Overrides a Trump Veto—for The First ...,701992,True,None,1610626466,False,1610626466,None
164,1355023622,bitcoinexchangeguide,1609511642,None,5819782141797507263,‌‌Congress Members Request Treasury to Extend ...,41151,True,None,1610572278,False,1610572278,None
165,1269687252,DeccanHeraldNews,1609487330,None,5453265223488128960,"TMC celebrates foundation day, Mamata Banerjee...",18368,True,None,1610568509,False,1610568509,None
166,1213817086,moneycontrolcom,1609480472,None,5213304687696029898,"As Trinamool Congress turns 23, we take a look...",10442,True,None,1610572877,False,1610572877,None


In [ ]:
traffic_data[traffic_data['date']>=1609455600]

In [39]:
traffic_data['date'].apply(datetime.datetime.fromtimestamp().strftime('%Y-%m-%d %H:%M:%S'))

TypeError: fromtimestamp() missing required argument 'timestamp' (pos 1)

In [ ]:
datetime.datetime.fromtimestamp(traffic_data['date'][1]).strftime('%Y-%m-%d %H:%M:%S')

In [53]:
df2 = traffic_data 
df2['date'] = pd.to_datetime(df2['date'], unit='s')

In [55]:
df2.to_csv('telegram_from_01-01-21.csv', encoding='UTF-8')